# Time series forecasting of Taipei's air quality index - dataset preprocessing

What follows are the preprocessing steps taken to obtain `df_ai`, the dataset used in the main notebook of this project. This dataset is built from those datasets on the [website of the Ministry of Environment of Taiwan](https://data.moenv.gov.tw/en/dataset/detail/AQX_P_488). Each of those datasets contain information for a single month on hourly Air Quality Index (AQI) values for multiple measuring sites of different Taiwanese cities and regions. By following the procedure that can be reproduced with this notebook's code, those datasets were combined and processed into a single dataset with the hourly information only for Taipei city, with each hourly value reflecting the average across the different measuring sites of the city. Some missing information is interpolated.

While extensive markdown explanation is absent, the comments within the code chunks should be enough to understand each step of the process.

### Load libraries

In [21]:
#load libraries for math, data wrangling and file reading
import numpy as np
import pandas as pd
import os

### Read and concatenate yearly air quality datasets

In [22]:
#read and concatenate individual datasets within the monthly_datasets folder
df_aqi = pd.concat([pd.read_csv(f'monthly_datasets/{f}') for f in os.listdir('monthly_datasets') if f.endswith('.csv')])

/var/folders/z4/rsx400qn1jldyy0yky98y40h0000gn/T/ipykernel_1117/1442458380.py:2: DtypeWarning: Columns (9,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_aqi = pd.concat([pd.read_csv(f'monthly_datasets/{f}') for f in os.listdir('monthly_datasets') if f.endswith('.csv')])
/var/folders/z4/rsx400qn1jldyy0yky98y40h0000gn/T/ipykernel_1117/1442458380.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_aqi = pd.concat([pd.read_csv(f'monthly_datasets/{f}') for f in os.listdir('monthly_datasets') if f.endswith('.csv')])
/var/folders/z4/rsx400qn1jldyy0yky98y40h0000gn/T/ipykernel_1117/1442458380.py:2: DtypeWarning: Columns (5,6,7,8,9,10,11,13,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_aqi = pd.concat([pd.read_csv(f'monthly_datasets/{f}') for f in os.listdir('monthly_datasets') if f.endswith('.csv')])
/var/folders/z4/rsx400qn1jldyy0yky98y40h0000gn/T/ipykern

In [23]:
#keep only relevant columns
df_aqi = df_aqi[['datacreationdate','county','sitename','aqi']]

#display the DataFrame with the relevant columns
display(df_aqi)

,datacreationdate,county,sitename,aqi
0,2019-08-01 00:00,高雄市,鳳山,22.0
1,2019-08-01 00:00,高雄市,大寮,43.0
2,2019-08-01 00:00,高雄市,林園,27.0
3,2019-08-01 00:00,高雄市,楠梓,47.0
4,2019-08-01 00:00,高雄市,左營,28.0
...,...,...,...,...
61915,2022-09-30 23:00,南投縣,南投,72.0
61916,2022-09-30 23:00,雲林縣,斗六,72.0
61917,2022-09-30 23:00,雲林縣,崙背,55.0
61918,2022-09-30 23:00,新北市,新北(樹林),48.0


### Select Taipei City information

In [24]:
#identify unique county names
unique_counties = df_aqi['county'].unique()

#display unique county names
display(unique_counties)

array(['高雄市', '屏東縣', '臺東縣', '雲林縣', '南投縣', '彰化縣', '臺中市', '苗栗縣', '新竹市',
       '新竹縣', '桃園市', '臺北市', '新北市', '嘉義縣', '嘉義市', '臺南市', '澎湖縣', '金門縣',
       '連江縣', '基隆市', '宜蘭縣', '花蓮縣', 'New Taipei City', 'Taipei City',
       'Taoyuan County', 'Hsinchu County', 'Hsinchu City',
       'Miaoli County', 'Taichung City', 'Changhua County',
       'Nantou County', 'Yunlin County', 'Chiayi County', 'Chiayi City',
       'Tainan City', 'Kaohsiung City', 'Pingtung County',
       'Taitung County', 'Hualien County', 'Yilan County', 'Keelung City',
       'Lienchiang County', 'Kinmen County', 'Penghu County', nan],
      dtype=object)

In [25]:
#create list with Taipei City name in Chinese and English and keep only those rows where county equals to one of those names
taipei_city_names = ['臺北市','Taipei City']
df_aqi = df_aqi[df_aqi.county.isin(taipei_city_names)]

#display length of DataFrame and make sure it is smaller than the original
display(df_aqi.shape)

(507371, 4)

### Unify site names

In [26]:
#identify unique site names, where each site name refers to a measuring site from which AQI readings were obtained
unique_sitenames = df_aqi['sitename'].unique()

#display unique site names
display(unique_sitenames)

array(['大同', '松山', '古亭', '萬華', '中山', '士林', '陽明', 'Shilin', 'Zhongshan',
       'Wanhua', 'Guting', 'Songshan', 'Datong', 'Yangming', '行動監測03'],
      dtype=object)

In [27]:
#create a dictionary mapping Chinese to English site names
site_name_mapping = {
    '大同': 'Datong',
    '松山': 'Songshan',
    '古亭': 'Guting',
    '萬華': 'Wanhua',
    '中山': 'Zhongshan',
    '士林': 'Shilin',
    '陽明': 'Yangming',
    '行動監測03': 'Mobile Monitoring 03'
}

#define function to unify site names into English
def unify_site_name(site_name):
    return site_name_mapping.get(site_name, site_name)

#apply the function to the sitename column
df_aqi['sitename'] = df_aqi['sitename'].apply(unify_site_name)

#get and display unique sitenames after unifying them, making sure no English-Chinese duplicity is present anymore
unique_sitenames = df_aqi['sitename'].unique()
display(unique_sitenames)

array(['Datong', 'Songshan', 'Guting', 'Wanhua', 'Zhongshan', 'Shilin',
       'Yangming', 'Mobile Monitoring 03'], dtype=object)

In [28]:
#get rid of county column, as all rows now correspond to the Taipei City county
df_aqi = df_aqi[['datacreationdate','sitename','aqi']]

### Handle duplicate rows

In [29]:
#get those rows that share the same datacreationdate and sitename values
duplicate_rows = df_aqi[df_aqi.duplicated(subset=['datacreationdate', 'sitename'], keep=False)]

#display sorted duplicate rows
duplicate_rows.sort_values(['datacreationdate', 'sitename'], ascending=[True, False])

,datacreationdate,sitename,aqi
37969,2018-01-21 18:00,Songshan,43.0
37982,2018-01-21 18:00,Songshan,43.0
37975,2018-01-21 18:00,Datong,35.0
37983,2018-01-21 18:00,Datong,35.0
21138,2020-11-11 17:00,Zhongshan,39.0
...,...,...,...
97873,2023-12-24 23:00,Shilin,48.0
97756,2023-12-24 23:00,Guting,52.0
97870,2023-12-24 23:00,Guting,52.0
97754,2023-12-24 23:00,Datong,60.0


In [30]:
#remove exact duplicates while keeping only the first occurrence
df_aqi = df_aqi.drop_duplicates(keep='first')

#to identify partial duplicates that may remain, get those rows that share the same datacreationdate and sitename values and display them sorted
duplicate_rows = df_aqi[df_aqi.duplicated(subset=['datacreationdate', 'sitename'], keep=False)]
duplicate_rows.sort_values(['datacreationdate', 'sitename'], ascending=[True, False])

,datacreationdate,sitename,aqi
34373,2020-11-20 01:00,Zhongshan,NaN
34464,2020-11-20 01:00,Zhongshan,52.0
44533,2020-11-22 13:00,Wanhua,32.0
44618,2020-11-22 13:00,Wanhua,33.0
45214,2020-11-22 17:00,Datong,NaN
...,...,...,...
118868,2021-06-30 23:00,Wanhua,40.0
8831,2022-03-03 10:00,Songshan,103.0
8947,2022-03-03 10:00,Songshan,NaN
9013,2022-03-03 11:00,Songshan,103.0


In [31]:
#remove those partial duplicate rows with NaN in the aqi column
df_aqi = df_aqi[df_aqi['aqi'].notna()]

#get those rows that share the same datacreationdate and sitename values and display them sorted
duplicate_rows = df_aqi[df_aqi.duplicated(subset=['datacreationdate', 'sitename'], keep=False)]
duplicate_rows.sort_values(['datacreationdate', 'sitename'], ascending=[True, False])

,datacreationdate,sitename,aqi
44533,2020-11-22 13:00,Wanhua,32.0
44618,2020-11-22 13:00,Wanhua,33.0
50174,2020-11-23 22:00,Datong,57.0
50198,2020-11-23 22:00,Datong,54.0
59358,2020-11-26 04:00,Zhongshan,32.0
...,...,...,...
118456,2021-06-30 21:00,Guting,41.0
118540,2021-06-30 22:00,Yangming,31.0
118688,2021-06-30 22:00,Yangming,30.0
118750,2021-06-30 23:00,Wanhua,39.0


In [32]:
#check how many aqi unique values we have within each group of rows with the same datacreationdate and sitename
duplicate_rows_grouped = duplicate_rows.groupby(['datacreationdate', 'sitename'])['aqi'].nunique()

#get and display the groups where there are different 'aqi' values
groups_with_different_aqi = duplicate_rows_grouped[duplicate_rows_grouped > 1]
display(groups_with_different_aqi)

datacreationdate  sitename 
2020-11-22 13:00  Wanhua       2
2020-11-23 22:00  Datong       2
2020-11-26 04:00  Zhongshan    2
2020-11-30 17:00  Guting       2
2020-12-02 05:00  Wanhua       2
                              ..
2021-06-30 19:00  Guting       2
2021-06-30 21:00  Guting       2
                  Songshan     2
2021-06-30 22:00  Yangming     2
2021-06-30 23:00  Wanhua       2
Name: aqi, Length: 9143, dtype: int64

In [33]:
#get indices of aforementioned groups and use them to filter the original duplicate rows to get only those with different 'aqi' values
groups_with_different_aqi_indices = duplicate_rows_grouped[duplicate_rows_grouped > 1].index
duplicates_with_different_aqi = duplicate_rows.set_index(['datacreationdate', 'sitename']).loc[groups_with_different_aqi_indices].reset_index()

#to solve divergence in the values, calculate the average 'aqi' for each group and round to the nearest integer
duplicates_averaged_aqi = duplicates_with_different_aqi.groupby(['datacreationdate', 'sitename'])['aqi'].mean().apply(np.ceil).astype(int).reset_index()

#remove the original duplicate rows with different aqi by creating a boolean mask and applying it
mask = df_aqi.set_index(['datacreationdate', 'sitename']).index.isin(duplicates_with_different_aqi.set_index(['datacreationdate', 'sitename']).index)
df_aqi = df_aqi[~mask] #apply the mask

#add the averaged rows back to the cleaned DataFrame
df_aqi = pd.concat([df_aqi, duplicates_averaged_aqi], ignore_index=True)

#get those rows that share the same datacreationdate and sitename values and display them sorted, expecting an empty DataFrame that will mean that no duplicates exist anymore
duplicate_rows = df_aqi[df_aqi.duplicated(subset=['datacreationdate', 'sitename'], keep=False)]
duplicate_rows.sort_values(['datacreationdate', 'sitename'], ascending=[True, False])

,datacreationdate,sitename,aqi


### Check if all values within each column conform to right format

In [34]:
#create DataFrame where we convert datacreationdate to datetime, coercing errors to NaT
df_aqi_formatcheck = df_aqi.copy()
df_aqi_formatcheck['datacreationdate2'] = pd.to_datetime(df_aqi_formatcheck['datacreationdate'], errors='coerce')

#display rows with NaT values in datacreationdate
wrong_format_dates = df_aqi_formatcheck[df_aqi_formatcheck['datacreationdate2'].isna()]
display(wrong_format_dates)

,datacreationdate,sitename,aqi,datacreationdate2
27381,2023/11/13 10:00:00,Guting,51.0,NaT
27382,2023/11/13 10:00:00,Wanhua,44.0,NaT
27383,2023/11/13 10:00:00,Zhongshan,40.0,NaT
27384,2023/11/13 10:00:00,Yangming,31.0,NaT
27385,2023/11/13 10:00:00,Shilin,35.0,NaT
27386,2023/11/13 10:00:00,Songshan,60.0,NaT
27387,2023/11/13 10:00:00,Datong,44.0,NaT


In [35]:
#since the different format is separation with '/' instead of ':', replace rows with wrong format by rows with good format
df_aqi.loc[df_aqi['datacreationdate'] == '2023/11/13 10:00:00', 'datacreationdate'] = '2020-11-22 13:00'

#repeat former steps again to check if bad format rows remain, where an empty DataFrame would mean no rows with bad format
df_aqi_formatcheck = df_aqi.copy()
df_aqi_formatcheck['datacreationdate2'] = pd.to_datetime(df_aqi_formatcheck['datacreationdate'], errors='coerce')
wrong_format_dates = df_aqi_formatcheck[df_aqi_formatcheck['datacreationdate2'].isna()]
display(wrong_format_dates)

,datacreationdate,sitename,aqi,datacreationdate2


### Calculate average air quality index across all sites for each time stamp

In [36]:
#group by 'datacreationdate' and calculate the average 'aqi' across all 'sitename'
df_aqi = df_aqi.groupby('datacreationdate')['aqi'].mean().round().astype(int).reset_index()

#display final DataFrame
display(df_aqi)

,datacreationdate,aqi
0,2016-11-25 13:00,27
1,2016-11-25 14:00,27
2,2016-11-25 15:00,27
3,2016-11-25 16:00,28
4,2016-11-25 17:00,28
...,...,...
66102,2024-06-30 19:00,34
66103,2024-06-30 20:00,33
66104,2024-06-30 21:00,33
66105,2024-06-30 22:00,32


### Check for missing hourly data and create it through interpolation

In [37]:
#convert datacreationdate to datetime, coercing errors to NaT
df_aqi['datacreationdate'] = pd.to_datetime(df_aqi['datacreationdate'], errors='coerce')

#create a complete range of hourly timestamps
full_time_range = pd.date_range(start="2016-11-25 13:00", end="2024-06-30 23:00", freq='h')

#identify missing timestamps by comparing with the datacreationtime column
missing_timestamps = full_time_range.difference(df_aqi['datacreationdate'])

#display missing timestamps and number of unique missing timestamps
print("Missing timestamps:")
print(missing_timestamps)

print("Number of unique missing timestamps:")
display(missing_timestamps.nunique())

Missing timestamps:
DatetimeIndex(['2016-11-28 19:00:00', '2016-11-28 20:00:00',
               '2016-11-28 21:00:00', '2016-11-28 22:00:00',
               '2016-11-28 23:00:00', '2016-11-29 00:00:00',
               '2016-11-29 01:00:00', '2016-11-29 02:00:00',
               '2016-11-29 03:00:00', '2016-11-29 04:00:00',
               ...
               '2024-01-07 09:00:00', '2024-02-18 14:00:00',
               '2024-05-13 05:00:00', '2024-05-13 10:00:00',
               '2024-06-24 05:00:00', '2024-06-24 06:00:00',
               '2024-06-24 07:00:00', '2024-06-24 08:00:00',
               '2024-06-24 09:00:00', '2024-06-24 10:00:00'],
              dtype='datetime64[ns]', length=480, freq=None)
Number of unique missing timestamps:


480

In [38]:
#check if timestamps for missing data are continuous by printing the missing data for a certain period
start_date = '2016-11-28 10:00:00'
end_date = '2016-11-28 23:00:00'
display(df_aqi[(df_aqi['datacreationdate'] >= start_date) & (df_aqi['datacreationdate'] <= end_date)])

,datacreationdate,aqi
69,2016-11-28 10:00:00,52
70,2016-11-28 11:00:00,54
71,2016-11-28 12:00:00,57
72,2016-11-28 13:00:00,77
73,2016-11-28 14:00:00,79
74,2016-11-28 15:00:00,81
75,2016-11-28 16:00:00,82
76,2016-11-28 17:00:00,83
77,2016-11-28 18:00:00,84


In [39]:
#set datacreationdate as the index and reindex the DataFrame to include all timestamps
df_aqi.set_index('datacreationdate', inplace=True)
df_aqi = df_aqi.reindex(full_time_range)

#copy the DataFrame before interpolation
df_aqi_original = df_aqi.copy()

#interpolate missing values and round interpolated values to nearest integer
df_aqi['aqi'] = df_aqi['aqi'].interpolate(method='time').round()

#create the "interpolated" column
df_aqi['interpolated'] = 0
df_aqi.loc[df_aqi_original['aqi'].isna() & df_aqi['aqi'].notna(), 'interpolated'] = 1

#reset the index to have datacreationdate as a column again
df_aqi.reset_index(inplace=True)
df_aqi.rename(columns={'index': 'datacreationdate'}, inplace=True)

#display the number of rows with interpolated = 1 
df_aqi['interpolated'].sum()

480

### Save final dataset

In [40]:
#rename datacreationdate column as time for clarity
df_aqi = df_aqi.rename(columns={"datacreationdate": "time"})

#save df_aqi to a CSV file
df_aqi.to_csv('df_aqi.csv', index=False)